# TODO  
## Card Similarity Search  
### Data Prep (raw card data > clean card data) 
  - ~~merge card, set, and legality data~~  
  - ~~concat token cards (later - may not want to)~~  
  - ensure clean  
    - fillna, non-english issues, maybe replace symbols ({T}...) w/ text  
  - **AWS**  
    - ~~Load raw MTGJson from S3~~  
    - ~~Lambda (or Glue): Prep > to S3~~  
### Similarity (clean card data > embeddings, similarity matrix)
  - USE embeddings from card text  
  - explore including other card props as text (color, type, mana cost...)  
  - similarity matrix  
  - pre-sort and save each card (50K+ cards)
  - **AWS**  
    - Load clean card data from S3  
    - SM Batch Job: USE embeddings & similarity matrix > to S3  
    - Lambda: embeddings/sim_matrix from S3 > EFS (?)  
    - Lambda: sort each card by similarity EFS > EFS (?)
    - Data Pipeline: cards data from S3 to Dynamo (?)  
### App Backend  
  - API accepts card name and returns topK similar cards (w/ some metadata for filtering)  
  - **AWS**  
    - Lambda: API queries EFS or Dynamo  
    - StepFunctions or Lambda destinations: refresh data pipeline as needed
### App Frontend  
  - Home page  
    - Search box and results  
    - A few filters (color, type, mana cost...)  
    - add placement  
    - sign in (eventually)  
### Deploy  
  - Backend
    - Serverless  
    - Seed  
  - Frontend  
    - React  
    - Amplify

# Data Prep

In [1]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

In [2]:
cards_df = pd.read_csv('../data/mtgjson/cards.csv')\
    .drop(columns=['index'])

print(cards_df.shape)
print('{} MB'.format(round(cards_df.memory_usage().sum()/1000000, 2)))
cards_df.head(1)

(55943, 74)
33.12 MB


,id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,...,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,1,Rebecca Guay,NaN,"mtgo,paper",black,123335.0,122967.0,G,NaN,G,...,NaN,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,38513fa0-ea83-5642-8ecd-4f0b3daa6768,NaN,NaN


In [3]:
sets_df = pd.read_csv('../data/mtgjson/sets.csv')[['code','name']]\
    .rename(columns={'name': 'setName', 'code':'setCode'})

print(sets_df.shape)
print('{} MB'.format(round(sets_df.memory_usage().sum()/1000000, 2)))
sets_df.head(1)

(545, 2)
0.01 MB


,setCode,setName
0,10E,Tenth Edition


In [4]:
# Merge set names into cards
cards_df = cards_df\
    .merge(sets_df, how='left', on='setCode')

print(cards_df.shape)
print('{} MB'.format(round(cards_df.memory_usage().sum()/1000000, 2)))
cards_df.head(1)

(55943, 75)
34.01 MB


,id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,...,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark,setName
0,1,Rebecca Guay,NaN,"mtgo,paper",black,123335.0,122967.0,G,NaN,G,...,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,38513fa0-ea83-5642-8ecd-4f0b3daa6768,NaN,NaN,Tenth Edition


In [5]:
legs_df = pd.read_csv('../data/mtgjson/legalities.csv')\
    .pivot(index='uuid', columns='format', values='status')\
    .reset_index()\
    .fillna('Blank')

print(legs_df.shape)
print('{} MB'.format(round(legs_df.memory_usage().sum()/1000000, 2)))
legs_df.head(1)

(54718, 14)
6.13 MB


format,uuid,brawl,commander,duel,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
0,00010d56-fe38-5e35-8aed-518019aa36a5,Blank,Legal,Legal,Blank,Blank,Legal,Legal,Blank,Blank,Blank,Legal,Blank,Legal


In [6]:
# Merge legalities into cards
cards_df = cards_df\
    .merge(legs_df, how='left', on='uuid')

print(cards_df.shape)
print('{} MB'.format(round(cards_df.memory_usage().sum()/1000000, 2)))
cards_df.head(1)

(55943, 88)
39.83 MB


,id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,...,future,historic,legacy,modern,oldschool,pauper,penny,pioneer,standard,vintage
0,1,Rebecca Guay,NaN,"mtgo,paper",black,123335.0,122967.0,G,NaN,G,...,Blank,Blank,Legal,Legal,Blank,Blank,Legal,Blank,Blank,Legal


In [34]:
cards_df.to_csv('cards/cards.csv', index=False)

### Handle token cards later

In [53]:
tokens_df = pd.read_csv('../data/mtgjson/tokens.csv')

print(tokens_df.shape)
print('{} MB'.format(round(tokens_df.memory_usage().sum()/1000000, 2)))
tokens_df.head(1)

(1704, 45)
0.61 MB


,index,id,artist,asciiName,availability,borderColor,colorIdentity,colors,edhrecRank,faceName,...,side,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,watermark
0,0,1,Jim Pavelec,NaN,paper,black,R,R,NaN,NaN,...,NaN,Dragon,NaN,78608.0,Flying,5,Token Creature — Dragon,"Token,Creature",7decf258-eb10-50da-83f7-c7eba74adbfb,NaN


In [3]:
print(cards_df.shape)
cards_df.head(2)

(55943, 74)


,id,artist,asciiName,availability,borderColor,cardKingdomFoilId,cardKingdomId,colorIdentity,colorIndicator,colors,...,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,1,Rebecca Guay,NaN,"mtgo,paper",black,123335.0,122967.0,G,NaN,G,...,NaN,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,38513fa0-ea83-5642-8ecd-4f0b3daa6768,NaN,NaN
1,2,Stephen Daniele,NaN,"mtgo,paper",black,123149.0,122781.0,U,NaN,U,...,"Human,Wizard",NaN,15024.0,When Academy Researchers enters the battlefiel...,2,Creature — Human Wizard,Creature,b8a68840-4044-52c0-a14e-0a1c630ba42c,NaN,NaN


In [55]:
cards_df.columns

Index(['id', 'artist', 'asciiName', 'availability', 'borderColor',
       'cardKingdomFoilId', 'cardKingdomId', 'colorIdentity', 'colorIndicator',
       'colors', 'convertedManaCost', 'duelDeck', 'edhrecRank',
       'faceConvertedManaCost', 'faceName', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isOnlineOnly', 'isOversized', 'isPromo', 'isReprint',
       'isReserved', 'isStarter', 'isStorySpotlight', 'isTextless',
       'isTimeshifted', 'keywords', 'layout', 'leadershipSkills', 'life',
       'loyalty', 'manaCost', 'mcmId', 'mcmMetaId', 'mtgArenaId',
       'mtgjsonV4Id', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'number',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'purchaseUrls', 'rarity',
       'scryfallId', 'scryfallIllustrationId', 'scryfallOracleI

In [7]:
cards_df.type.nunique()

1971

In [8]:
cards_df.setCode.nunique()

531

In [9]:
cards_df.memory_usage().sum()/1000000

33.118384

In [56]:
for col in cards_df.columns:
    print(col + ': ' + str(cards_df[col][0]) + '\n')

id: 1

artist: Rebecca Guay

asciiName: nan

availability: mtgo,paper

borderColor: black

cardKingdomFoilId: 123335.0

cardKingdomId: 122967.0

colorIdentity: G

colorIndicator: nan

colors: G

convertedManaCost: 4.0

duelDeck: nan

edhrecRank: 1111.0

faceConvertedManaCost: nan

faceName: nan

flavorName: nan

flavorText: nan

frameEffects: nan

frameVersion: 2003

hand: nan

hasAlternativeDeckLimit: 0


hasFoil: 1

hasNonFoil: 1

isAlternative: 0

isFullArt: 0

isOnlineOnly: 0

isOversized: 0

isPromo: 0

isReprint: 1

isReserved: 0

isStarter: 0

isStorySpotlight: 0

isTextless: 0

isTimeshifted: 0

keywords: nan

layout: normal

leadershipSkills: nan

life: nan

loyalty: nan

manaCost: {2}{G}{G}

mcmId: 16413.0

mcmMetaId: 19.0

mtgArenaId: nan

mtgjsonV4Id: 1669af17-d287-5094-b005-4b143441442f

mtgoFoilId: 27283.0

mtgoId: 27282.0

multiverseId: 130483.0

name: Abundance

number: 249

originalReleaseDate: nan

originalText: If you would draw a card, you may instead choose land or

# Download and Store USE Model from TFHub

In [5]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


## Saved the downloaded USE-Large model

In [6]:
tf.saved_model.save(model, "../models/use-large")

INFO:tensorflow:Assets written to: ../models/use-large\assets
INFO:tensorflow:Assets written to: ../models/use-large\assets


## Load USE-Large from local disk

In [16]:
use_embed = hub.KerasLayer('../models/use-large')

***
# Get USE Embeddings

In [7]:
arena_df = cards_df.query('mtgArenaId.notnull()')\
    .reset_index(drop=True)\
    .fillna(value={'text': 'Blank'})

In [8]:
arena_df[['uuid']].memory_usage().sum()

43480

In [9]:
arena_txt = list(arena_df.text)
arena_txt[0]

'Flying, lifelink\nPegasus creatures you control have lifelink.\nConstellation — Whenever an enchantment enters the battlefield under your control, create a 2/2 white Pegasus creature token with flying.'

In [25]:
arena_name = [(name + '-' + set_name).replace(' ','_') for name, set_name in zip(arena_df.name, arena_df.setCode)]
arena_name[0]

"Archon_of_Sun's_Grace-AJMP"

In [14]:
arena_txt[23]

'Blank'

In [17]:
embeddings = use_embed(arena_txt)
print(embeddings.shape)

(5419, 512)


In [21]:
embeddings.numpy().size * embeddings.numpy().itemsize

20480

In [21]:
corr = np.inner(embeddings, embeddings)
print(corr.shape)

(5419, 5419)


In [34]:
import plotly.express as px

In [26]:
card_df = pd.DataFrame(corr, columns=arena_name, index=arena_name)
card_df.head()

,Archon_of_Sun's_Grace-AJMP,Audacious_Thief-AJMP,Banishing_Light-AJMP,Bond_of_Revival-AJMP,Carnifex_Demon-AJMP,Doomed_Necromancer-AJMP,Dryad_Greenseeker-AJMP,Fanatic_of_Mogis-AJMP,"Gadwick,_the_Wizened-AJMP",Goblin_Oriflamme-AJMP,...,Veteran_Adventurer-ZNR,Vine_Gecko-ZNR,Wayward_Guide-Beast-ZNR,Windrider_Wizard-ZNR,"Yasharn,_Implacable_Earth-ZNR","Zagras,_Thief_of_Heartbeats-ZNR","Zareth_San,_the_Trickster-ZNR",Zof_Consumption_//_Zof_Bloodbog-ZNR,Zof_Consumption_//_Zof_Bloodbog-ZNR,Zulaport_Duelist-ZNR
Archon_of_Sun's_Grace-AJMP,1.000000,0.388027,0.540133,0.626441,0.514713,0.497357,0.414993,0.587006,0.667409,0.597402,...,0.372712,0.545192,0.698247,0.554204,0.654835,0.693039,0.552946,0.199106,0.365390,0.576491
Audacious_Thief-AJMP,0.388027,1.000000,0.364457,0.447599,0.297466,0.377156,0.339213,0.379261,0.452389,0.415212,...,0.386959,0.421869,0.409472,0.473218,0.464308,0.392080,0.448119,0.156082,0.608094,0.399204
Banishing_Light-AJMP,0.540133,0.364457,1.000000,0.583113,0.458850,0.504666,0.293982,0.581625,0.707416,0.389173,...,0.250604,0.421688,0.535110,0.520927,0.534659,0.525379,0.570414,0.242288,0.342716,0.529742
Bond_of_Revival-AJMP,0.626441,0.447599,0.583113,1.000000,0.434825,0.724121,0.501235,0.621211,0.629852,0.517640,...,0.403822,0.546920,0.636535,0.627735,0.653420,0.622629,0.633170,0.261863,0.468044,0.594412
Carnifex_Demon-AJMP,0.514713,0.297466,0.458850,0.434825,1.000000,0.416845,0.235222,0.385975,0.495592,0.379345,...,0.288310,0.477718,0.459487,0.346581,0.424836,0.498399,0.440100,0.355213,0.344039,0.480329


In [29]:
card_df[["Archon_of_Sun's_Grace-AJMP"]].sort_values(by="Archon_of_Sun's_Grace-AJMP", ascending=False)

,Archon_of_Sun's_Grace-AJMP
Archon_of_Sun's_Grace-AJMP,1.000000
Archon_of_Sun's_Grace-THB,1.000000
"Alela,_Artful_Provocateur-ELD",0.868950
Ethereal_Absolution-RNA,0.832532
Depose_//_Deploy-RNA,0.817002
...,...
Clearwater_Pathway_//_Murkwater_Pathway-ZNR,-0.019223
Island-ANA,-0.019223
Wind_Strider-XLN,-0.023939
Living_Tempest-ZNR,-0.023939


In [107]:
test_card = 'Golos,_Tireless_Pilgrim'
test_card

'Golos,_Tireless_Pilgrim'

In [106]:
[card for card in card_df.columns if card.startswith('Golos')]

['Golos,_Tireless_Pilgrim']

In [108]:
card_df[[test_card]].sort_values(by=test_card, ascending=False)

,"Golos,_Tireless_Pilgrim"
3059,1.000000
1687,0.805584
1583,0.801413
2606,0.798853
835,0.796900
...,...
5170,-0.020036
3215,-0.020036
5212,-0.041830
4441,-0.041830


In [109]:
test_card = test_card.replace('_',' ')
arena_df.query('name == @test_card').text.values

array(['When Golos, Tireless Pilgrim enters the battlefield, you may search your library for a land card, put that card onto the battlefield tapped, then shuffle your library.\n{2}{W}{U}{B}{R}{G}: Exile the top three cards of your library. You may play them this turn without paying their mana costs.'],
      dtype=object)

In [112]:
test_name = arena_name[1583].replace('_',' ')
test_name

'Emergent Ultimatum'

In [113]:
arena_df.query('name == @test_name').text.values

array(['Search your library for up to three monocolored cards with different names and exile them. An opponent chooses one of those cards. Shuffle that card into your library. You may cast the other cards without paying their mana costs. Exile Emergent Ultimatum.'],
      dtype=object)

In [52]:
fig = px.imshow(card_df)
fig.show()